# Test Impact Matrix Generation

## Objectif
Après l'exécution de test200.py, un fichier nommé simulations.csv est généré, contenant des données d'échantillons basées sur une quantité fixe de matériau de 1kg. L'objectif est maintenant de générer une nouvelle matrice avec la même structure (caractéristiques et catégories d'impact), mais avec des valeurs mises à jour qui reflètent les quantités réelles de matériaux utilisées et leurs facteurs de renouvellement pendant la durée de vie du bâtiment.

## 1. Importation des bibliothèques nécessaires

In [87]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import sys

# Importer la classe QUACI de test101.py
sys.path.append(os.path.dirname(os.getcwd()))
from test101 import QUACI

# Configuration pour l'affichage
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
pd.set_option('display.width', 1000)

## 2. Chargement du fichier simulations.csv

Ce fichier contient les impacts environnementaux pour 1kg de chaque matériau.

In [88]:
# Chargement du fichier simulations.csv
simulations_path = 'simulations.csv'

try:
    # Chargement du fichier d'origine
    original_df = pd.read_csv(simulations_path)
    print(f"Fichier {simulations_path} chargé avec succès.")
    print(f"Dimensions: {original_df.shape}")
    
    # Vérification du format: si la première colonne est 'Catégorie d'impact', alors le format est transposé
    is_transposed = original_df.columns[0] == 'Catégorie d\'impact'
    
    if is_transposed:
        # Conversion du format: les matériaux sont en colonnes, les catégories en lignes
        print("\nFormat détecté: matériaux en colonnes et catégories d'impact en lignes.")
        print("Conversion du format pour traitement...")
        
        # Définir la première colonne comme index
        indexed_df = original_df.set_index('Catégorie d\'impact')
        
        # Transposer le dataframe pour obtenir les matériaux en lignes
        transposed_df = indexed_df.transpose().reset_index()
        
        # Renommer la colonne d'index en 'material'
        transposed_df = transposed_df.rename(columns={'index': 'material'})
        
        # Utiliser ce dataframe transposé comme simulations_df
        simulations_df = transposed_df
        print("\nConversion terminée. Format maintenant: matériaux en lignes et catégories d'impact en colonnes.")
    else:
        # Format standard, utiliser tel quel
        simulations_df = original_df
        print("\nFormat standard détecté: matériaux en lignes et catégories d'impact en colonnes.")
except FileNotFoundError:
    print(f"Erreur: Le fichier {simulations_path} n'a pas été trouvé. Exécutez test200.py d'abord.")
    simulations_df = None
except Exception as e:
    print(f"Erreur lors du chargement du fichier: {str(e)}")
    simulations_df = None

Fichier simulations.csv chargé avec succès.
Dimensions: (27, 24)

Format détecté: matériaux en colonnes et catégories d'impact en lignes.
Conversion du format pour traitement...

Conversion terminée. Format maintenant: matériaux en lignes et catégories d'impact en colonnes.


### Exploration des données de simulation

In [83]:
# Afficher les premières lignes pour comprendre la structure
if simulations_df is not None:
    print("Aperçu des données de simulation:")
    simulations_df.head()

Aperçu des données de simulation:


In [84]:
# Identifier les matériaux et les catégories d'impact
if simulations_df is not None:
    # Extraction des informations
    if 'material' in simulations_df.columns:
        # Format transposé ou standard
        materials = simulations_df['material'].unique().tolist()
        impact_categories = [col for col in simulations_df.columns if col != 'material']
        
        print("\nMatériaux identifiés:")
        for material in materials:
            print(f"  - {material}")
        
        print(f"\nNombre de catégories d'impact: {len(impact_categories)}")
        print("Exemples de catégories d'impact:")
        for impact in impact_categories[:5]:  # Afficher les 5 premiers pour ne pas surcharger
            print(f"  - {impact}")
        if len(impact_categories) > 5:
            print(f"  ... et {len(impact_categories) - 5} autres catégories")
    else:
        print("\nStructure de fichier non reconnue. Assurez-vous que le fichier contient une colonne 'material' ou 'Catégorie d\'impact'.")


Matériaux identifiés:
  - ExternalWood
  - BeamWood
  - Cinderblock
  - Concrete
  - Earthen
  - Firedbrick
  - Glass
  - Glass wool
  - Gypsum
  - Hemp
  - Lath
  - Mortar
  - OSB
  - Paint
  - Plywood
  - Steel
  - WaterProofing
  - XPS
  - External Wood OSB
  - Poutrelle
  - Beam
  - Parquet
  - Glazing
  - Wool
  - Polystyrene
  - Aluminum
  - PV Systems
  - Battery
  - HVAC
  - DHW
  - Fired Bricks
  - Earth

Nombre de catégories d'impact: 27
Exemples de catégories d'impact:
  - Acidification
  - Climate change
  - Climate change - Biogenic
  - Climate change - Fossil
  - Climate change - Land use and LU change
  ... et 22 autres catégories


## 3. Chargement des données de quantité de matériaux

Charger les quantités de matériaux depuis le fichier Stage_QUACI-ACV.xlsx

In [85]:
# Chemin vers le fichier Excel contenant les quantités de matériaux
quantities_path = os.path.join('Calculating House Matrices', 'Stage_QUACI-ACV.xlsx')

try:
    # Charger le tableau des matériaux et leurs quantités
    quantities_df = pd.read_excel(quantities_path, sheet_name='Feuil1', header=1)
    print(f"Fichier {quantities_path} chargé avec succès.")
except FileNotFoundError:
    print(f"Erreur: Le fichier {quantities_path} n'a pas été trouvé.")
    quantities_df = None
except Exception as e:
    print(f"Erreur lors de la lecture du fichier {quantities_path}: {str(e)}")
    quantities_df = None

Erreur lors de la lecture du fichier Calculating House Matrices\Stage_QUACI-ACV.xlsx: Worksheet named 'Feuil1' not found


### Exploration des données de quantités de matériaux

In [72]:
# Afficher les données de quantités
if quantities_df is not None:
    print("Aperçu des données de quantités de matériaux:")
    quantities_df.head()

In [73]:
# Extraction des durées de vie des matériaux
if quantities_df is not None:
    # Explorer la structure du fichier pour extraire les durées de vie
    print("Colonnes disponibles:")
    print(quantities_df.columns.tolist())
    
    # Supposons que la première ligne contient les durées de vie (à ajuster selon la structure réelle)
    if 'Durée de vie' in quantities_df.columns:
        materials_lifetime = quantities_df.iloc[0, 1:].to_dict()
        print("\nDurées de vie des matériaux:")
        for material, lifetime in materials_lifetime.items():
            print(f"{material}: {lifetime} ans")

## 4. Interface utilisateur pour saisir les paramètres

Dans cette section, vous allez entrer la durée de vie de la maison et les durées de vie des matériaux utilisés. Le facteur de renouvellement sera calculé comme le rapport entre la durée de vie de la maison et celle de chaque matériau.

In [74]:
# Saisie de la durée de vie de la maison
house_lifecycle = int(input("Entrez la durée de vie de la maison (en années): "))
print(f"\nDurée de vie de la maison: {house_lifecycle} ans")

# Chargement des données des récapitulatifs pour vérification
recap_folder_path = 'Material_Statistics'
available_recap_files = []

if os.path.exists(recap_folder_path):
    available_recap_files = [f for f in os.listdir(recap_folder_path) if f.endswith('récap.XLSX') or f.endswith('recap.xlsx')]
    if available_recap_files:
        print(f"\nTableaux récapitulatifs disponibles dans '{recap_folder_path}':")
        for file in available_recap_files:
            print(f"  - {file}")
    else:
        print(f"\nAucun tableau récapitulatif trouvé dans '{recap_folder_path}'")


Durée de vie de la maison: 50 ans

Tableaux récapitulatifs disponibles dans 'Material_Statistics':
  - BeamWood_Tableau récap.XLSX
  - Cinderblock_Tableau récap.XLSX
  - Concrete_Tableau récap.XLSX
  - Erathen_Tableau récap.XLSX
  - ExternalWood_Tableau récap.XLSX
  - Firedbrick_Tableau récap.XLSX
  - Glass wool_Tableau récap.XLSX
  - Glass_Tableau récap.XLSX
  - Gypsum_Tableau récap.XLSX
  - Hemp_Tableau récap.XLSX
  - Lath_Tableau récap.XLSX
  - Mortar_Tableau récap.XLSX
  - OSB_Tableau récap.XLSX
  - Paint_Tableau récap.XLSX
  - Plywood_Tableau récap.XLSX
  - Steel_Tableau récap.XLSX
  - WaterProofing_Tableau récap.XLSX
  - XPS_Tableau récap.XLSX


In [75]:
# Options pour la saisie des durées de vie des matériaux
print("\nVous avez plusieurs options pour entrer les durées de vie des matériaux:")
print("1. Utiliser les valeurs par défaut du fichier Excel (si disponible)")
print("2. Modifier les durées de vie manuellement")
print("3. Importer depuis les tableaux récapitulatifs des matériaux")

choice = input("\nChoisissez une option (1, 2 ou 3): ")

# Dictionnaire pour stocker les durées de vie des matériaux
material_lifetimes = {}

# Option 1: Utiliser les valeurs par défaut
if choice == "1":
    if quantities_df is not None and 'Durée de vie' in quantities_df.columns:
        # Utiliser les durées de vie extraites du fichier Excel comme valeurs par défaut
        material_lifetimes = quantities_df.iloc[0, 1:].to_dict()
        print("\nDurées de vie par défaut utilisées:")
        for material, lifetime in material_lifetimes.items():
            print(f"{material}: {lifetime} ans")
    else:
        print("\nAucune donnée de durée de vie par défaut disponible dans le fichier Excel.")
        # Utiliser les valeurs standard définies dans backend.py
        standard_lifetimes = {
            "ExternalW.OSB": 50, "Poutrelle": 50, "Beam": 50, "Parquet": 25,
            "Steel": 50, "Glazing": 25, "Wood": 50, "WaterProof": 15,
            "Polystyren": 20, "Gypsum": 30, "Aluminum": 50, "Paint": 10,
            "Mortar": 50, "PV": 25, "Battery": 10, "HVAC": 20,
            "DHW": 15, "CinderBlock": 100, "FiredBrick": 100, "Earth": 100,
            "Hemp": 50, "Concrete": 100, "Glass": 25, "OSB": 50,
            "Plywood": 50, "Glass Wool": 50, "XPS": 50, "Lath": 50
        }
        material_lifetimes = standard_lifetimes
        print("\nUtilisation des durées de vie standard:")
        for material, lifetime in sorted(material_lifetimes.items()):
            print(f"{material}: {lifetime} ans")

# Option 2: Modifier manuellement
elif choice == "2":
    # Vérifier si nous avons des matériaux identifiés
    materials_to_ask = []
    
    # Priorité 1: Matériaux du fichier simulations.csv
    if simulations_df is not None and 'material' in simulations_df.columns:
        materials_to_ask = simulations_df['material'].unique().tolist()
        print(f"\nUtilisation des {len(materials_to_ask)} matériaux identifiés dans simulations.csv")
    # Sinon, essayer d'extraire les matériaux des colonnes si le format est transposé
    elif 'original_df' in locals() and is_transposed:
        materials_to_ask = [col for col in original_df.columns if col != "Catégorie d'impact"]
        print(f"\nUtilisation des {len(materials_to_ask)} matériaux identifiés dans les colonnes de simulations.csv")
    # Sinon, utiliser les données du fichier Excel si disponibles
    elif quantities_df is not None:
        materials_to_ask = quantities_df.columns[1:].tolist()  # Ignorer la première colonne (labels)
        print(f"\nUtilisation des {len(materials_to_ask)} matériaux identifiés dans le fichier Excel")
    
    if materials_to_ask:
        # Valeurs standard comme références
        standard_lifetimes = {
            "ExternalW.OSB": 50, "Poutrelle": 50, "Beam": 50, "Parquet": 25,
            "Steel": 50, "Glazing": 25, "Wood": 50, "WaterProof": 15,
            "Polystyren": 20, "Gypsum": 30, "Aluminum": 50, "Paint": 10,
            "Mortar": 50, "PV": 25, "Battery": 10, "HVAC": 20,
            "DHW": 15, "CinderBlock": 100, "FiredBrick": 100, "Earth": 100,
            "Hemp": 50, "Concrete": 100, "Glass": 25, "OSB": 50,
            "Plywood": 50, "Glass Wool": 50, "XPS": 50, "Lath": 50,
            "ExternalWood": 50, "BeamWood": 50, "Cinderblock": 100, "Firedbrick": 100, "WaterProofing": 15
        }
        
        print("\nSaisissez la durée de vie pour chaque matériau:")
        for material in materials_to_ask:
            # Essayer de trouver une valeur par défaut
            default_lifetime = None
            for key, value in standard_lifetimes.items():
                if key.lower().replace(" ", "").replace(".", "") in material.lower().replace(" ", "").replace(".", "") or \
                   material.lower().replace(" ", "").replace(".", "") in key.lower().replace(" ", "").replace(".", ""):
                    default_lifetime = value
                    break
            
            # Si aucune correspondance n'est trouvée, utiliser une durée de vie générique basée sur le type de matériau
            if default_lifetime is None:
                if any(term in material.lower() for term in ['steel', 'metal', 'aluminum', 'aluminium']):
                    default_lifetime = 50
                elif any(term in material.lower() for term in ['wood', 'beam', 'osb', 'plywood', 'lath']):
                    default_lifetime = 50
                elif any(term in material.lower() for term in ['concrete', 'brick', 'block', 'cinder']):
                    default_lifetime = 100
                elif any(term in material.lower() for term in ['glass']):
                    default_lifetime = 25
                elif any(term in material.lower() for term in ['paint']):
                    default_lifetime = 10
                elif any(term in material.lower() for term in ['waterproof']):
                    default_lifetime = 15
                elif any(term in material.lower() for term in ['gypsum']):
                    default_lifetime = 30
                elif any(term in material.lower() for term in ['insulation', 'wool', 'xps', 'hemp']):
                    default_lifetime = 50
                else:
                    default_lifetime = 50  # Valeur par défaut
            
            # Demander à l'utilisateur avec la valeur par défaut
            user_input = input(f"Durée de vie de {material} (défaut: {default_lifetime} ans): ")
            if user_input.strip():
                try:
                    material_lifetimes[material] = int(user_input)
                except ValueError:
                    print(f"Valeur non valide, utilisation de la valeur par défaut: {default_lifetime} ans")
                    material_lifetimes[material] = default_lifetime
            else:
                material_lifetimes[material] = default_lifetime
    else:
        print("\nImpossible de déterminer les matériaux pour la saisie manuelle des durées de vie.")
        # Utiliser les valeurs standard comme solution de repli
        standard_lifetimes = {
            "ExternalW.OSB": 50, "Poutrelle": 50, "Beam": 50, "Parquet": 25,
            "Steel": 50, "Glazing": 25, "Wood": 50, "WaterProof": 15,
            "Polystyren": 20, "Gypsum": 30, "Aluminum": 50, "Paint": 10,
            "Mortar": 50, "PV": 25, "Battery": 10, "HVAC": 20,
            "DHW": 15, "CinderBlock": 100, "FiredBrick": 100, "Earth": 100,
            "Hemp": 50, "Concrete": 100, "Glass": 25, "OSB": 50,
            "Plywood": 50, "Glass Wool": 50, "XPS": 50, "Lath": 50
        }
        material_lifetimes = standard_lifetimes

# Option 3: Importer depuis les tableaux récapitulatifs - reste inchangée


Vous avez plusieurs options pour entrer les durées de vie des matériaux:
1. Utiliser les valeurs par défaut du fichier Excel (si disponible)
2. Modifier les durées de vie manuellement
3. Importer depuis les tableaux récapitulatifs des matériaux

Utilisation des 17 matériaux identifiés dans simulations.csv

Saisissez la durée de vie pour chaque matériau:

Utilisation des 17 matériaux identifiés dans simulations.csv

Saisissez la durée de vie pour chaque matériau:


## 5. Calcul du facteur de renouvellement

Pour chaque matériau, calcul du facteur de renouvellement comme :
Facteur de renouvellement = (Durée de vie de la maison) / (Durée de vie du matériau)

In [76]:
# Calcul des facteurs de renouvellement en utilisant la logique de la classe QUACI
renewal_factors = {}

for material, lifetime in material_lifetimes.items():
    if lifetime > 0:
        # Utiliser np.maximum pour s'assurer que le facteur est toujours >= 1
        renewal_factors[material] = np.maximum(1.0, house_lifecycle / lifetime)
    else:
        renewal_factors[material] = 1.0  # Valeur par défaut sécurisée

# Afficher les facteurs de renouvellement calculés
print("Facteurs de renouvellement calculés:")
for material, factor in renewal_factors.items():
    print(f"{material}: {factor:.2f}")

# Vérifier si tous les facteurs sont >= 1
min_factor = min(renewal_factors.values()) if renewal_factors else 1.0
if min_factor < 1.0:
    print("\nATTENTION: Certains facteurs de renouvellement sont inférieurs à 1.")
    print("Les facteurs devraient être >= 1, car un matériau doit être remplacé au moins une fois.")
    print("Correction automatique des facteurs...")
    
    # Corriger les facteurs < 1
    for material in renewal_factors:
        renewal_factors[material] = max(1.0, renewal_factors[material])
    
    print("\nFacteurs de renouvellement corrigés:")
    for material, factor in renewal_factors.items():
        print(f"{material}: {factor:.2f}")

Facteurs de renouvellement calculés:
ExternalWood: 1.00
BeamWood: 1.00
Cinderblock: 1.00
Concrete: 1.00
Firedbrick: 1.00
Glass: 1.00
Glass wool: 1.00
Gypsum: 1.25
Hemp: 1.25
Lath: 1.25
Mortar: 1.25
OSB: 1.25
Paint: 1.25
Plywood: 1.25
Steel: 1.25
WaterProofing: 1.25
XPS: 1.00


## 6. Extraction et préparation des quantités de matériaux et calcul avec QUACI

Cette section extrait les quantités de matériaux du fichier Excel et utilise la classe QUACI pour calculer les facteurs de renouvellement.

In [ ]:
# Extraction des quantités de matériaux
material_quantities = {}

if quantities_df is not None:
    # Trouver la ligne qui contient les quantités (en supposant qu'elle est dans la ligne "Facteur de Quantité")
    quantity_row = None
    for i, row in quantities_df.iterrows():
        if 'Facteur de Quantité' in str(row.iloc[0]) or 'quantité' in str(row.iloc[0]).lower():
            quantity_row = row
            break
    
    if quantity_row is not None:
        # Extraire les quantités par matériau
        for col in quantities_df.columns[1:]:  # Ignorer la première colonne qui est probablement un libellé
            material_quantities[col] = float(quantity_row[col]) if pd.notna(quantity_row[col]) else 0
        
        print("Quantités de matériaux extraites:")
        for material, quantity in material_quantities.items():
            print(f"{material}: {quantity}")
    else:
        print("Impossible de trouver la ligne contenant les quantités de matériaux.")
else:
    print("Données de quantités non disponibles.")

# Utilisation directe de la classe QUACI pour calculer les facteurs de renouvellement et quantités finales
if material_quantities:
    # Préparation des données pour QUACI
    comp_quantity = pd.Series(material_quantities)
    dur_vie_comp = pd.Series({mat: material_lifetimes.get(mat, 50) for mat in material_quantities})
    
    # Création d'une instance QUACI
    quaci_instance = QUACI(
        comp_quantity=comp_quantity,
        dur_vie_mean=float(house_lifecycle),
        dur_vie_std_dev=0.0,  # Sans écart-type pour un calcul déterministe
        dur_vie_comp=dur_vie_comp,
        path='Material_Statistics'
    )
    
    # Récupération des facteurs de renouvellement calculés par QUACI
    print("\nFacteurs de renouvellement calculés par QUACI:")
    for material in quaci_instance.comp_quantity.index:
        factor = quaci_instance.fact_renouv.loc[material]
        renewal_factors[material] = factor  # Mise à jour des facteurs de renouvellement
        print(f"{material}: {factor:.2f}")
    
    print("\nQuantités finales calculées par QUACI (quantité × facteur):")
    for material in quaci_instance.final_comp_quantity.index:
        final_quantity = quaci_instance.final_comp_quantity.loc[material]
        print(f"{material}: {final_quantity:.2f} kg")
else:
    # Si pas de quantités disponibles, utiliser les facteurs déjà calculés
    print("\nUtilisation des facteurs de renouvellement calculés précédemment.")

Données de quantités non disponibles.

Facteurs de renouvellement calculés par QUACI:

Quantités finales calculées par QUACI (quantité × facteur):

Facteurs de renouvellement calculés par QUACI:

Quantités finales calculées par QUACI (quantité × facteur):


## 7. Mise à l'échelle des valeurs de simulation

Multipliez les valeurs originales de simulations.csv par :
- La quantité totale du matériau
- Le facteur de renouvellement pour ce matériau

In [78]:
# Fonction améliorée pour faire correspondre les noms des matériaux entre les sources de données
def match_material_names(simulation_material, material_quantities):
    """Faire correspondre un nom de matériau de la simulation avec les noms des quantités."""
    # Simplification des noms pour la correspondance
    sim_material_simple = simulation_material.lower().replace(" ", "").replace("-", "").replace("_", "")
    
    # Correspondance exacte
    for mat in material_quantities.keys():
        if mat.lower().replace(" ", "").replace("-", "").replace("_", "") == sim_material_simple:
            return mat
    
    # Correspondance partielle
    for mat in material_quantities.keys():
        mat_simple = mat.lower().replace(" ", "").replace("-", "").replace("_", "")
        if sim_material_simple in mat_simple or mat_simple in sim_material_simple:
            return mat
    
    # Correspondances spécifiques pour certains matériaux courants
    material_mapping = {
        "externalwood": ["external", "wood", "externw"],
        "beamwood": ["beam", "wood"],
        "cinderblock": ["cinder", "block", "concrete block"],
        "firedbrick": ["fired", "brick", "brique"],
        "glass wool": ["glasswool", "laine", "insulation"],
        "waterproofing": ["waterproof", "water proof", "étanchéité"]
    }
    
    for key, synonyms in material_mapping.items():
        if any(synonym in sim_material_simple for synonym in synonyms):
            for mat in material_quantities.keys():
                if key in mat.lower().replace(" ", ""):
                    return mat
    
    print(f"Avertissement: Aucune correspondance trouvée pour '{simulation_material}'")
    return None

In [79]:
# Création d'une copie de simulations_df pour les valeurs mises à l'échelle
scaled_impacts_df = None
successful_scaling = False

if simulations_df is not None:
    # Vérifier si nous avons les données nécessaires pour la mise à l'échelle
    if not material_quantities:
        print("\nAvertissement: Aucune donnée de quantité de matériaux disponible.")
        print("Les valeurs d'impact ne peuvent pas être mises à l'échelle sans quantités.")
        print("Vérifiez le fichier Stage_QUACI-ACV.xlsx ou entrez les quantités manuellement.")
        # Demander à l'utilisateur s'il veut entrer les quantités manuellement
        manual_input = input("Voulez-vous entrer les quantités manuellement? (oui/non): ").lower() == 'oui'
        
        if manual_input:
            material_quantities = {}
            # Si le format original est transposé, utiliser les noms des colonnes comme matériaux
            if 'original_df' in locals() and is_transposed:
                materials_list = [col for col in original_df.columns if col != "Catégorie d'impact"]
            # Sinon, utiliser la colonne 'material' du DataFrame transformé
            elif 'material' in simulations_df.columns:
                materials_list = simulations_df['material'].unique().tolist()
            else:
                materials_list = []
                print("Impossible d'identifier les matériaux pour la saisie des quantités.")
            
            # Demander à l'utilisateur d'entrer les quantités pour chaque matériau
            for material in materials_list:
                try:
                    quantity = float(input(f"Quantité de {material} (en kg): "))
                    material_quantities[material] = quantity
                except ValueError:
                    print(f"Valeur non valide pour {material}, utilisation de 1.0 kg par défaut")
                    material_quantities[material] = 1.0
    
    if not renewal_factors:
        print("\nAvertissement: Les facteurs de renouvellement sont manquants.")
        print("Utilisation de facteurs de renouvellement par défaut.")
        # Utiliser des facteurs de renouvellement par défaut (minimum 1.0)
        renewal_factors = {material: 1.0 for material in material_quantities}
    
    # Création d'une copie pour stocker les valeurs mises à l'échelle
    if 'material' in simulations_df.columns:
        scaled_impacts_df = simulations_df.copy()
        impact_columns = [col for col in scaled_impacts_df.columns if col != 'material']
        
        # Variables pour suivre la progression
        successful_matches = 0
        failed_matches = 0
        
        print("\nDébut de la mise à l'échelle des impacts par matériau...")
        
        # Parcourir chaque ligne (matériau)
        for idx, row in scaled_impacts_df.iterrows():
            material = row['material']
            matched_material = match_material_names(material, material_quantities)
            
            if matched_material:
                quantity = material_quantities[matched_material]
                renewal = renewal_factors.get(matched_material, 1.0)  # Minimum 1.0
                scale_factor = quantity * renewal
                
                print(f"Mise à l'échelle de {material} (correspond à {matched_material}): "
                      f"Quantité={quantity}, Renouvellement={renewal:.2f}, Facteur total={scale_factor:.2f}")
                
                # Mise à l'échelle des valeurs d'impact
                for col in impact_columns:
                    scaled_impacts_df.loc[idx, col] = row[col] * scale_factor
                
                successful_matches += 1
            else:
                # Si aucune correspondance n'est trouvée, essayer d'utiliser le nom tel quel
                if material in material_quantities:
                    quantity = material_quantities[material]
                    renewal = renewal_factors.get(material, 1.0)  # Minimum 1.0
                    scale_factor = quantity * renewal
                    
                    print(f"Mise à l'échelle de {material} (correspondance directe): "
                          f"Quantité={quantity}, Renouvellement={renewal:.2f}, Facteur total={scale_factor:.2f}")
                    
                    # Mise à l'échelle des valeurs d'impact
                    for col in impact_columns:
                        scaled_impacts_df.loc[idx, col] = row[col] * scale_factor
                    
                    successful_matches += 1
                else:
                    print(f"Impossible de mettre à l'échelle {material} : aucune correspondance trouvée")
                    # Conserver les valeurs d'origine pour les matériaux sans correspondance
                    # mais avec un facteur de renouvellement par défaut d'au moins 1.0
                    default_renewal = max(1.0, house_lifecycle / 50)  # Durée de vie par défaut de 50 ans, minimum 1.0
                    print(f"  → Utilisation d'un facteur de mise à l'échelle par défaut: {default_renewal:.2f} (basé sur 50 ans)")
                    
                    for col in impact_columns:
                        scaled_impacts_df.loc[idx, col] = row[col] * default_renewal
                    
                    failed_matches += 1
        
        # Résumé de la mise à l'échelle
        total_materials = successful_matches + failed_matches
        success_rate = (successful_matches / total_materials) * 100 if total_materials > 0 else 0
        
        print(f"\nRésumé de la mise à l'échelle:")
        print(f"  - Matériaux traités avec succès: {successful_matches}/{total_materials} ({success_rate:.1f}%)")
        print(f"  - Matériaux sans correspondance exacte: {failed_matches}/{total_materials} ({100-success_rate:.1f}%)")
        
        if success_rate >= 50:
            successful_scaling = True
            print("\nLa mise à l'échelle a été effectuée avec succès pour la majorité des matériaux.")
        else:
            print("\nAttention: La mise à l'échelle a échoué pour la majorité des matériaux.")
            print("Vérifiez les correspondances entre les noms de matériaux.")
            
            # Demander à l'utilisateur s'il veut continuer malgré le faible taux de succès
            continue_anyway = input("Voulez-vous quand même continuer et générer la matrice? (oui/non): ").lower() == 'oui'
            if continue_anyway:
                successful_scaling = True
    else:
        print("\nErreur: Le DataFrame ne contient pas de colonne 'material'.")
        print("La mise à l'échelle ne peut pas être effectuée.")


Avertissement: Aucune donnée de quantité de matériaux disponible.
Les valeurs d'impact ne peuvent pas être mises à l'échelle sans quantités.
Vérifiez le fichier Stage_QUACI-ACV.xlsx ou entrez les quantités manuellement.

Début de la mise à l'échelle des impacts par matériau...
Mise à l'échelle de ExternalWood (correspond à ExternalWood): Quantité=123.0, Renouvellement=1.00, Facteur total=123.00
Mise à l'échelle de BeamWood (correspond à BeamWood): Quantité=123.0, Renouvellement=1.00, Facteur total=123.00
Mise à l'échelle de Cinderblock (correspond à Cinderblock): Quantité=123.0, Renouvellement=1.00, Facteur total=123.00
Mise à l'échelle de Concrete (correspond à Concrete): Quantité=123.0, Renouvellement=1.00, Facteur total=123.00
Mise à l'échelle de Firedbrick (correspond à Firedbrick): Quantité=123.0, Renouvellement=1.00, Facteur total=123.00
Mise à l'échelle de Glass (correspond à Glass): Quantité=123.0, Renouvellement=1.00, Facteur total=123.00
Mise à l'échelle de Glass wool (corre

## 8. Génération de la matrice finale

In [80]:
# Personnaliser le nom de fichier et générer la matrice d'impacts mise à l'échelle
if scaled_impacts_df is not None and (successful_scaling or 'continue_anyway' in locals() and continue_anyway):
    # Demander à l'utilisateur le nom du fichier de sortie
    default_output = f"impacts_scaled_house_{house_lifecycle}years.csv"
    output_filename = input(f"Nom du fichier de sortie (défaut: {default_output}): ")
    
    # Si l'utilisateur n'a rien entré, utiliser la valeur par défaut
    if not output_filename.strip():
        output_filename = default_output
    
    # S'assurer que le fichier a une extension .csv
    if not output_filename.lower().endswith('.csv'):
        output_filename += '.csv'
    
    # S'assurer que le nom du fichier est différent de simulations.csv
    if output_filename.lower() == 'simulations.csv':
        print("ATTENTION: Le nom du fichier de sortie ne peut pas être 'simulations.csv'. ")
        output_filename = f"impacts_scaled_{house_lifecycle}years.csv"
        print(f"Nom de fichier modifié à: {output_filename}")
    
    # Ajouter un horodatage si le fichier existe déjà
    if os.path.exists(output_filename):
        import datetime
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        name_part, ext_part = os.path.splitext(output_filename)
        output_filename = f"{name_part}_{timestamp}{ext_part}"
        print(f"Le fichier existait déjà. Nouveau nom: {output_filename}")
    
    try:
        # Générer une version transposée similaire au format d'origine de simulations.csv si nécessaire
        if 'original_df' in locals() and is_transposed:
            # Convertir la matrice mise à l'échelle au format d'origine (catégories en lignes, matériaux en colonnes)
            # Définir la colonne material comme index
            matrix_for_save = scaled_impacts_df.set_index('material')
            # Transposer pour avoir les catégories en lignes et matériaux en colonnes
            matrix_for_save = matrix_for_save.transpose().reset_index()
            # Renommer la colonne d'index en 'Catégorie d'impact'
            matrix_for_save = matrix_for_save.rename(columns={'index': "Catégorie d'impact"})
            # Enregistrer dans ce format
            matrix_for_save.to_csv(output_filename, index=False)
            print(f"\nMatrice d'impacts mise à l'échelle (format transposé) enregistrée dans: {output_filename}")
        else:
            # Format standard (matériaux en lignes, catégories en colonnes)
            scaled_impacts_df.to_csv(output_filename, index=False)
            print(f"\nMatrice d'impacts mise à l'échelle enregistrée dans: {output_filename}")
            
        print("Cette matrice contient les impacts environnementaux mis à l'échelle selon les quantités réelles")
        print(f"de matériaux et leurs facteurs de renouvellement sur {house_lifecycle} ans.")
        
        # Afficher le chemin absolu du fichier généré pour faciliter l'accès
        full_path = os.path.abspath(output_filename)
        print(f"\nChemin complet du fichier: {full_path}")
    except Exception as e:
        print(f"\nErreur lors de l'enregistrement de la matrice: {str(e)}")
        # Tentative de sauvegarde avec un nom de fichier alternatif
        fallback_filename = f"matrix_backup_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        try:
            scaled_impacts_df.to_csv(fallback_filename, index=False)
            print(f"Sauvegarde de secours réussie dans: {fallback_filename}")
        except:
            print("Échec de la sauvegarde de secours.")
else:
    print("\nImpossible de générer la matrice finale: aucune donnée mise à l'échelle disponible.")
    print("Vérifiez les étapes précédentes et assurez-vous que la mise à l'échelle a été effectuée correctement.")

Le fichier existait déjà. Nouveau nom: impacts_scaled_house_50years_20250504_221907.csv

Matrice d'impacts mise à l'échelle (format transposé) enregistrée dans: impacts_scaled_house_50years_20250504_221907.csv
Cette matrice contient les impacts environnementaux mis à l'échelle selon les quantités réelles
de matériaux et leurs facteurs de renouvellement sur 50 ans.

Chemin complet du fichier: c:\Users\MSI\Documents\GitHub\QUACI-ACV\impacts_scaled_house_50years_20250504_221907.csv


## 9. Visualisation des résultats

In [35]:
# Comparaison des impacts avant et après mise à l'échelle
if 'scaled_impacts_df' in locals() and 'material' in scaled_impacts_df.columns:
    # Sélectionner un impact pour la visualisation (par exemple, le premier impact dans la liste)
    impact_columns = [col for col in scaled_impacts_df.columns if col != 'material']
    if impact_columns:
        impact_to_visualize = impact_columns[0]
        
        # Vérifier si les colonnes nécessaires existent dans simulations_df
        if impact_to_visualize in simulations_df.columns:
            # Créer un graphique pour comparer les impacts avant et après mise à l'échelle
            plt.figure(figsize=(12, 8))
            
            original_impacts = simulations_df.set_index('material')[impact_to_visualize]
            scaled_impacts = scaled_impacts_df.set_index('material')[impact_to_visualize]
            
            # Vérifier les valeurs manquantes et les gérer
            original_impacts = original_impacts.fillna(0)
            scaled_impacts = scaled_impacts.fillna(0)
            
            # Calcul des rapports pour affichage sur l'étiquette
            scaling_ratios = scaled_impacts / original_impacts
            scaling_ratios = scaling_ratios.replace([np.inf, -np.inf], 0).fillna(0)  # Gérer les divisions par zéro
            
            # Création du graphique
            ax = scaled_impacts.plot(kind='bar', color='coral', alpha=0.7, label='Après mise à l\'échelle')
            original_impacts.plot(kind='bar', color='skyblue', alpha=0.7, label='Original (1kg)', ax=ax)
            
            plt.title(f'Comparaison des impacts {impact_to_visualize} - Avant et après mise à l\'échelle')
            plt.xlabel('Matériau')
            plt.ylabel(f'Impact {impact_to_visualize}')
            plt.xticks(rotation=45, ha='right')
            plt.legend()
            plt.grid(axis='y', linestyle='--', alpha=0.7)
            plt.tight_layout()
            
            # Ajouter des annotations pour les facteurs de mise à l'échelle
            for i, material in enumerate(original_impacts.index):
                if material in scaling_ratios.index:
                    plt.text(i, scaled_impacts[material], 
                            f"x{scaling_ratios[material]:.1f}", 
                            ha='center', va='bottom', rotation=0)
            
            plt.show()
        else:
            print(f"Erreur : La colonne '{impact_to_visualize}' n'existe pas dans simulations_df.")

In [36]:
# Visualisation de la contribution des matériaux à l'impact total pour chaque catégorie
if 'scaled_impacts_df' in locals() and 'material' in scaled_impacts_df.columns:
    impact_columns = [col for col in scaled_impacts_df.columns if col != 'material']
    
    # Préparation des données pour la visualisation
    total_impacts = scaled_impacts_df[impact_columns].sum()
    
    # Sélection des 3 catégories d'impact les plus importantes
    top_impacts = total_impacts.sort_values(ascending=False).head(3).index.tolist()
    
    # Création de graphiques pour les principales catégories d'impact
    for impact in top_impacts:
        plt.figure(figsize=(10, 6))
        
        # Trier par contribution décroissante
        sorted_data = scaled_impacts_df.sort_values(by=impact, ascending=False)
        
        # Calculer la contribution relative en pourcentage
        total = sorted_data[impact].sum()
        relative_contribution = sorted_data[impact] / total * 100
        
        # Créer un graphique à barres
        bars = plt.bar(sorted_data['material'], sorted_data[impact], color='skyblue')
        
        # Ajouter des étiquettes avec les pourcentages
        for i, bar in enumerate(bars):
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2., height,
                    f'{relative_contribution.iloc[i]:.1f}%',
                    ha='center', va='bottom', rotation=0)
        
        plt.title(f'Contribution des matériaux à l\'impact {impact}')
        plt.xlabel('Matériaux')
        plt.ylabel(f'Impact {impact}')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        
        plt.show()

## 10. Analyse comparative des alternatives (si disponible)

Cette section pourrait être développée si vous avez plusieurs alternatives de conception à comparer.

In [37]:
# Code pour l'analyse comparative des alternatives
# Ce code serait développé si vous avez plusieurs alternatives de conception

## Conclusion

Cette analyse nous a permis de générer une matrice d'impacts environnementaux mise à l'échelle qui reflète les quantités réelles de matériaux utilisés dans le projet et prend en compte leurs facteurs de renouvellement pendant la durée de vie du bâtiment. Ces résultats peuvent être utilisés pour l'analyse du cycle de vie (ACV) du bâtiment et comparer différentes alternatives de conception.